In [26]:
%matplotlib inline

In [27]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [28]:
def RGB_to_HSV(img):
    b,g,r = cv2.split(img)
    rg = cv2.merge([r,g,b])
    rg = rg.astype(np.float32)

    np.seterr(all = 'ignore')
    mask = np.empty_like(rg)
    arr_max = rg.max(-1)
    delta = rg.ptp(-1)
    s = delta/arr_max
    s[delta==0]=0
    idx = (rg[:,:,0] == arr_max)
    mask[idx,0] = np.true_divide((rg[idx,1] - rg[idx,2]), delta[idx])
    idx = (rg[:,:,1] == arr_max)
    mask[idx,0] = 2. + np.true_divide((rg[idx,2] - rg[idx,0]), delta[idx])
    idx = (rg[:,:,2] == arr_max)
    mask[idx,0] = 4. + np.true_divide((rg[idx,0] - rg[idx,1]), delta[idx])
    mask[:,:,0] = ((mask[:,:,0]/6.0)%1.0)*255.0
    mask[:,:,0] = np.nan_to_num(mask[:,:,0])
    mask[:,:,1] = s * 255.0
    mask[:,:,2] = arr_max
    mask = mask.astype(np.uint8)
    
    return mask

In [29]:
def Quality_Control(img):
    #convert to hsv
    mask = RGB_to_HSV(img)
    
    points = cv2.inRange(mask, (0, 0, 0), (255,255,165))
    c, _ = cv2.findContours(points, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in c:
        if cv2.contourArea(cnt) > 20000:
            return 'YES'
        elif cv2.contourArea(cnt) > 10000:
            return 'MAYBE'
        
    return 'NO'

In [30]:
def Analyze(img):
    dot_img = img.copy()
    # sharpening the image
    image = cv2.GaussianBlur(img, (0,0), 3)
    img = cv2.addWeighted(img, 1.5, image, -0.5, 0)
    # color thresholding
    yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    im_th = cv2.inRange(yuv, (0,0,0), (150,255,255))

    cells = []
    cell_area = 0
    # ------- finding contours in the image --------
    c, _ = cv2.findContours(im_th, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    for cnt in c:
        area = cv2.contourArea(cnt)
        true_area = cv2.contourArea(cnt, True)
        x,y,w,h = cv2.boundingRect(cnt)
        aspect_ratio = float(w)/h
        # keeping only the "valid" cells
        if area > 500 and aspect_ratio > 0.6 and area < 20000 and true_area < 0: 
            cv2.drawContours(img, [cnt], -1, (255,0,0), 3)
            cell_area += area
            cells.append(cnt)
            
    # --------- finding dots in the image -----------------------
    mask = RGB_to_HSV(dot_img)

    # thresholding the dots found
    dot_points = cv2.inRange(mask, (0, 50, 0), (255,255,150))
    c, heirarchy = cv2.findContours(dot_points, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)

    dots = []
    # keep only the dot contours larger than 10 px
    for cnt in c:
        if cv2.contourArea(cnt) > 10:
            dots.append(cnt)

    cv2.drawContours(img, dots, -1, (0,0,255),2)
    
    
    total = {}
    track = -1
    # -------- doing the point polygon test to check dot was within a cell ----------
    for i in range(len(dots)):
        for j in range(len(dots[i])):
            for m in range(len(cells)):
                x = dots[i][j][0][0]
                y = dots[i][j][0][1]
                points = (x,y)
                num  = cv2.pointPolygonTest(cells[m], points, False )
                if num == 0 or num == 1:
                    moment = cv2.moments(cells[m])
                    if moment['m00'] != 0.0:
                        cx = moment['m10']/moment['m00']
                        cy = moment['m01']/moment['m00']
                        centroid = (cx,cy)
                        if centroid in total and track != i:
                            total[centroid] += 1
                            track += 1
                        elif centroid not in total and track != i:
                            total[centroid] = 1
                            track += 1

    average = 0
    for k,v in total.iteritems():
        average += v

    if len(total) != 0.0:
        dot_average = average / len(total)
    else:
        dot_average = 0
            
    return (str(len(cells)), str(cell_area), str(dot_average))

In [31]:
def main():
    file_path = os.listdir("Images/")
    yellow_test = open('Yellow_Test_Final.txt', 'w')
    yellow_test.write("Image\t# Of Cells\tTotal Cell Area\tDots per Cell\tFlagged\n")
    for infile in file_path:
        if "Ki67" in infile:
            img = cv2.imread('Images/'+infile)
            flagged = Quality_Control(img)
            output = Analyze(img)
            yellow_test.write(infile+'\t'+output[0]+'\t'+output[1]+'\t'+output[2]+'\t'+flagged+'\n')
    yellow_test.close()
main()